In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import datetime, pathlib, time

# Default download directory for Windows
downloads = pathlib.Path.home() / 'Downloads'
destination = pathlib.Path.cwd()
now = datetime.datetime.now()

def smash_grab(report, sevis_name, save_name):
    matching_files = str(report + '_matching_files')
    most_recent = str(report + '_most_recent')
    most_recent_str_dttm = str(report + '_most_recent_str_dttm')
    most_recent_dttm = str(report + '_most_recent_dttm')
    report_destination = str(report + '_destination')
    matching_files = []
    destination_file_name = str(str(now.year) + '_' + str(now.month) + '_' + str(now.day) + '_' + str(save_name))
    for file in list(downloads.glob(sevis_name)):
        matching_files.append(str(datetime.datetime.fromtimestamp(file.stat().st_ctime)) + '|' + str(file))
    matching_files.sort(reverse=True)
    most_recent = pathlib.Path(matching_files[0].split('|')[1])
    most_recent_str_dttm = str(pathlib.Path(matching_files[0].split('|')[0]))
    most_recent_dttm = datetime.datetime.strptime(most_recent_str_dttm, '%Y-%m-%d %H:%M:%S.%f')
    if most_recent_dttm > now:
        print('Most recent file successfully copied: ' + str(most_recent_dttm > now))
        report_destination = destination / destination_file_name
        if not report_destination.exists():
            most_recent.replace(str(report_destination))
    else:
        raise Exception('File not found')
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 10)
def holdup(id_value):
    wait.until(EC.element_to_be_clickable((By.ID, id_value)))

# Change to match your campus(es); the key is your preferred campus name, the value is the name as it appears in SEVIS
campuses = {'CAMPUS_A': '[title*="SEVIS_CAMPUS_A_NAME"]',
            'CAMPUS_B': '[title*="SEVIS_CAMPUS_B_NAME"]',
            'CAMPUS_C': '[title*="SEVIS_CAMPUS_C_NAME"]',
            'CAMPUS_D': '[title*="SEVIS_CAMPUS_D_NAME"]',
            'CAMPUS_E': '[title*="SEVIS_CAMPUS_E_NAME"]'}

driver.get('https://egov.ice.gov/sevis/action/userLogon')

# Login
holdup('username')
username = driver.find_element_by_id('username')
username.clear()
# DSO's SEVIS username
username.send_keys('USERNAME')
username.send_keys(Keys.RETURN)
password = driver.find_element_by_id('password')
password.clear()
# DSO's SEVIS password
password.send_keys('PASSWORD')
password.send_keys(Keys.RETURN)

# Continue past warnings
holdup('submitButton')
submit = driver.find_element_by_id('submitButton')
submit.send_keys(Keys.RETURN)

holdup('submit')
read_understand = driver.find_element_by_id('submit')
read_understand.send_keys(Keys.RETURN)

for key in campuses:
    # Select Campus; you may need to change this value to fit your situation
    holdup('school6090')
    campus_select = driver.find_element_by_css_selector(campuses[key])
    campus_select.send_keys(Keys.SPACE)
    # Navigate to Student Lists, Transferring In
    campus_lists = driver.find_element_by_id('studentListButton')
    campus_lists.send_keys(Keys.SPACE)
    campus_lists.send_keys(Keys.RETURN)
    holdup('studentList')
    student_xfer_in = driver.find_element_by_xpath('//a[@href="/sevis/action/eligibility/StudentList?student_list=transfer_in_active_status"]')
    student_xfer_in.send_keys(Keys.SPACE)
    student_xfer_in.send_keys(Keys.RETURN)
    # Display max entries, download CSV
    holdup('studentListTable_transfer_in_active_status_lengthId')
    time.sleep(5)
    list_entries = Select(driver.find_element_by_id('studentListTable_transfer_in_active_status_lengthId'))
    list_entries.select_by_value('1000')
    list_csv = driver.find_element_by_xpath('//a[@class="dt-button buttons-csv buttons-html5"]')
    list_csv.send_keys(Keys.SPACE)
    list_csv.send_keys(Keys.RETURN)
    # Wait a few
    time.sleep(5)
    try:
        smash_grab(str(key + 'xfer-in'), 'SEVIS - Students Transferring In*.csv', str('xfer-in-' + key +'.csv'))
    except:
        pass
    # Return to Main
    main = driver.find_element_by_xpath('//a[@href="/sevis/action/common/MainPageData"]')
    main.send_keys(Keys.SPACE)
    main.send_keys(Keys.RETURN)
    #(Skip Campus selection); you may need to change this value to fit your situation
    holdup('school6090')
    downloads_button = driver.find_element_by_id('downloadButton')
    downloads_button.send_keys(Keys.SPACE)
    downloads_button.send_keys(Keys.RETURN)
    # Navigate to Downloads
    holdup('downloadReportStatusCSV')
    dso_report = driver.find_element_by_id('downloadReportStatusCSV')
    dso_report.send_keys(Keys.SPACE)
    dso_report.send_keys(Keys.RETURN)
    holdup('status')
    dso_report_popup = driver.find_element_by_id('status')
    dso_report_popup.send_keys('i')
    action = ActionChains(driver)
    action.send_keys(Keys.TAB)
    action.send_keys(Keys.TAB)
    action.send_keys(Keys.SPACE)
    action.send_keys(Keys.RETURN)
    action.perform()
    # Wait a few
    time.sleep(5)
    try:
        smash_grab(str(key + 'initial'), 'REPORT_STUDENT_STATUS*.csv', str('initial_status_' + key + '.csv'))
    except:
        action = ActionChains(driver)
        action.send_keys(Keys.SPACE)
        action.send_keys(Keys.RETURN)
        action.perform()
    # Return to Main
    go_back = driver.find_element_by_xpath('//a[@href="/sevis/action/common/MainPageData"]')
    go_back.send_keys(Keys.SPACE)
    go_back.send_keys(Keys.RETURN)
    
# Goodbye
main = driver.find_element_by_xpath('//a[@href="/sevis/action/logoff"]')
main.send_keys(Keys.SPACE)
main.send_keys(Keys.RETURN)
driver.quit()